In [1]:
import pandas as pd
import json
import numpy as np
import requests
import pandas_profiling
from pandas_profiling.utils.cache import cache_file
from ast import literal_eval
from pathlib import Path
from datetime import datetime

In [2]:
#Import data from csv files 
start=datetime.now()

Credits = pd.read_csv('credits.csv', dtype={"cast": "string", "crew": str, "id": int})
Keywords = pd.read_csv('keywords.csv')
Links = pd.read_csv('links_small.csv')
Metadata = pd.read_csv('movies_metadata.csv',
                 lineterminator='\n', dtype = np.dtype('unicode'))
Ratings = pd.read_csv('ratings_small.csv')

print (datetime.now()-start)

0:00:02.179592


In [3]:
print(Credits.info())
print(Keywords.info())
print(Links.info())
print(Metadata.info())
print(Ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  string
 1   crew    45476 non-null  object
 2   id      45476 non-null  int32 
dtypes: int32(1), object(1), string(1)
memory usage: 888.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
me

In [4]:

print(Metadata[~Metadata["id"].astype(int).apply(lambda x: isinstance(x, (int)))])

#Metadata[Metadata["id"] == "2014-01-01"]


Metadata["id"] = Metadata["id"].astype(int)

print(Metadata.info())

Empty DataFrame
Columns: [adult, belongs_to_collection, budget, genres, homepage, id, imdb_id, original_language, original_title, overview, popularity, poster_path, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, video, vote_average, vote_count]
Index: []

[0 rows x 24 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   adult                  45463 non-null  object
 1   belongs_to_collection  4491 non-null   object
 2   budget                 45463 non-null  object
 3   genres                 45463 non-null  object
 4   homepage               7779 non-null   object
 5   id                     45463 non-null  int32 
 6   imdb_id                45446 non-null  object
 7   original_language      45452 non-null  object
 8   original_title         4546

In [5]:
# Merge the Movies metadata and prepare for further formatting
start=datetime.now()


Metadata = Metadata.merge(Credits, on='id')
Movies_dataset = Metadata.merge(Keywords, on='id')

Movies_dataset.shape

Movies_dataset['cast'] = Movies_dataset['cast'].apply(literal_eval)
Movies_dataset['crew'] = Movies_dataset['crew'].apply(literal_eval)
Movies_dataset['keywords'] = Movies_dataset['keywords'].apply(literal_eval)
Movies_dataset['genres'] = Movies_dataset['genres'].apply(literal_eval)

Movies_dataset['year'] = pd.DatetimeIndex(Movies_dataset['release_date']).year

print (datetime.now()-start)

print(Movies_dataset.info())

0:00:31.913404
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  46628 non-null  object 
 1   belongs_to_collection  4574 non-null   object 
 2   budget                 46628 non-null  object 
 3   genres                 46628 non-null  object 
 4   homepage               8009 non-null   object 
 5   id                     46628 non-null  int32  
 6   imdb_id                46611 non-null  object 
 7   original_language      46617 non-null  object 
 8   original_title         46628 non-null  object 
 9   overview               45633 non-null  object 
 10  popularity             46628 non-null  object 
 11  poster_path            46233 non-null  object 
 12  production_companies   46628 non-null  object 
 13  production_countries   46628 non-null  object 
 14  release_date           46544 non-null  

In [6]:
# Isolate the Director and Novel info
start=datetime.now()

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

Movies_dataset['director'] = Movies_dataset['crew'].apply(get_director)

def get_novel(x):
    for i in x:
        if i['job'] == 'Novel':
            return 'Yes'#i['name']
    return 'No'

Movies_dataset['novel'] = Movies_dataset['crew'].apply(get_novel)

print (datetime.now()-start)

0:00:00.141921


In [7]:
#print(Movies_dataset.info())
print(Movies_dataset['cast'][0][0])

{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}


In [8]:
# Split the cast and store in separate dataframe
start=datetime.now()

cast_dataset = pd.DataFrame()
cast_cols = ['movie_id','order', 'gender','name']

def split_cast(x, y, cols):
    cast = pd.DataFrame()
    
    for j in y:
        df2 = pd.DataFrame([[x,j['order'],j['gender'], j['name']]], columns = cols)
        cast = cast.append(df2)
    return cast


cast_dataset = pd.concat(Movies_dataset.apply(lambda x: split_cast(x['id'], x['cast'], cast_cols), axis=1).tolist())

print (datetime.now()-start)

0:10:16.037490


In [9]:

print(cast_dataset.head())
print(cast_dataset.info())
print(cast_dataset.groupby(by='movie_id').count())

   movie_id  order  gender           name
0       862      0       2      Tom Hanks
0       862      1       2      Tim Allen
0       862      2       2    Don Rickles
0       862      3       2     Jim Varney
0       862      4       2  Wallace Shawn
<class 'pandas.core.frame.DataFrame'>
Int64Index: 574164 entries, 0 to 0
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   movie_id  574164 non-null  int64 
 1   order     574164 non-null  int64 
 2   gender    574164 non-null  int64 
 3   name      574164 non-null  object
dtypes: int64(3), object(1)
memory usage: 21.9+ MB
None
          order  gender  name
movie_id                     
2             4       4     4
3             7       7     7
5            24      24    24
6            15      15    15
11          106     106   106
...         ...     ...   ...
464207        9       9     9
465044        6       6     6
467731       14      14    14
468707        6       6

In [10]:
# Split the Keywords and store in separate dataframe
start=datetime.now()

def split_keywords(x,y):
    
    keywords = pd.json_normalize(y)
    keywords['movie_id'] = x

    if keywords.empty: return
    else: return keywords[['name', 'movie_id']]

keywords_dataset = pd.concat(Movies_dataset.apply(lambda x: split_keywords(x['id'],x['keywords']), axis = 1).tolist())

print (datetime.now()-start)

0:01:15.853712


In [11]:
print(keywords_dataset.head())
print(keywords_dataset.info())
print(keywords_dataset.groupby(by='movie_id').count())

         name  movie_id
0    jealousy       862
1         toy       862
2         boy       862
3  friendship       862
4     friends       862
<class 'pandas.core.frame.DataFrame'>
Int64Index: 159441 entries, 0 to 2
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   name      159441 non-null  object
 1   movie_id  159441 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
None
          name
movie_id      
2            7
3            4
5           10
6            5
11          16
...        ...
463800       2
463906       1
464207       3
464819       1
468707       3

[31092 rows x 1 columns]


In [12]:
# split the genres in individual flags in the dataset
start=datetime.now()

genre_dataset = pd.DataFrame()
genre_cols = ['id', 'IsAnimation', 'IsComedy', 'IsFamily', 'IsThriller', 'IsRomance', 'IsDrama', 'IsAdventure']

def classify_genre(x, cols):
    genre = pd.DataFrame()
    for i, row in x.iterrows():
        
        df2 = pd.DataFrame([[row['id'],False, False, False, False, False, False, False]], columns = cols)
        
        for j in row['genres']:
            '''animation = False
            comedy = False
            family = False
            thriller = False
            romance = False
            drama = False'''
            
            if j['name'] == 'Animation':
                df2['IsAnimation'] = True
            elif j['name'] == 'Comedy':
                df2['IsComedy'] = True
            elif j['name'] == 'Family':
                df2['IsFamily'] = True 
            elif j['name'] == 'Thriller':
                df2['IsThriller'] = True            
            elif j['name'] == 'Romance':
                df2['IsRomance'] = True
            elif j['name'] == 'Drama':
                df2['IsDrama'] = True
            elif j['name'] == 'Adventure':
                df2['IsAdventure'] = True
        genre = genre.append(df2)
    return genre

genre_dataset = classify_genre(Movies_dataset[['id', 'genres']], genre_cols)
#print(genre_dataset)


# Merge the genre dataset with the movie dataset, excluding the combined version of 'genres' column
Movies_dataset = Movies_dataset[Movies_dataset.columns.difference(['genres'])].merge(genre_dataset, on='id')

print (datetime.now()-start)

0:01:19.999700


In [13]:

Movies_dataset.info()
Movies_dataset[0:1]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50914 entries, 0 to 50913
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  50914 non-null  object 
 1   belongs_to_collection  4940 non-null   object 
 2   budget                 50914 non-null  object 
 3   cast                   50914 non-null  object 
 4   crew                   50914 non-null  object 
 5   director               49937 non-null  object 
 6   homepage               8689 non-null   object 
 7   id                     50914 non-null  int32  
 8   imdb_id                50897 non-null  object 
 9   keywords               50914 non-null  object 
 10  novel                  50914 non-null  object 
 11  original_language      50903 non-null  object 
 12  original_title         50914 non-null  object 
 13  overview               49837 non-null  object 
 14  popularity             50914 non-null  object 
 15  po

,adult,belongs_to_collection,budget,cast,crew,director,homepage,id,imdb_id,keywords,...,vote_average,vote_count,year,IsAnimation,IsComedy,IsFamily,IsThriller,IsRomance,IsDrama,IsAdventure
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",John Lasseter,http://toystory.disney.com/toy-story,862,tt0114709,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",...,7.7,5415,1995.0,True,True,True,False,False,False,False


In [14]:
#profile_report = Movies_dataset.profile_report(explorative=True, html={'style': {'full_width': True}})
#profile_report.to_widgets()